---

## Tugas Besar II AI: Prediksi Income Per Tahun
#### IF3170 Inteligensi Buatan

| Nama        | NIM           |
| ------------- |:-------------:|
| Farhan Makarim      | 13515003     |
| Taufan Mahaputra      | 13515028 |
| Rizki Halasan      | 13515095     |
| Erfandi Suryo Putra      | 135150145     |
---

## Membaca data income 

In [1]:
from numpy import genfromtxt
import numpy as np
import pandas as pd
from pandas import read_csv 
from sklearn import preprocessing
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import Normalizer


dataset = read_csv('CencusIncome.data.txt', index_col=False,
                names=['age', 'workclass', 'fnlwght', 'education', 'education-num', 'marital-status',
                                                          'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
                                                          'hours-per-week', 'native-country', 'income'])



## Melakukan penanganan terhadap data

## Hasil analisis data yang dilakukan adalah kami menemukan bahwa atribut fnlwght, hours-per-week, dan education-num bisa dihapus

### Atribut fnlwght bisa dihapus karena atribut tersebut merupakan penurunan dari beberapa atribut lainnya
 
### Atribut hours-per-week bisa dihapus karena atribut tersebut dapat menyebabkan bias pada penghasilan (orang yang memiliki hours-per-week tinggi belum tentu penghasilannya besar, tergantung jenis pekerjaannya)

### Atribut education-num bisa dihapus karena atribut tersebut berhubungan dengan atribut education

## Selain menghapus beberapa atribut, kami juga mengisi nilai '?' dengan modus pada atribut tersebut. Nilai '?' terdapat pada 3 atribut, yaitu workclass, native-country, dan 'occupation'

### Nilai '?' pada atribut workclass diganti dengan ' Private'
### Nilai '?' pada atribut native country diganti dengan ' United-States'
### Nilai '?' pada atribut occupation sempat kami ganti dengan ' Prof-specialty', namun karena mengurangi akurasi, kami tidak jadi menggantinya

In [2]:
# Mengubah data yang berbentuk string menjadi bentuk integer 
# supaya dapat diolah scikit learn
le = preprocessing.LabelEncoder()

# missing values diganti dengan data mayoritas
for i in range(32561):
    if(dataset['workclass'][i] == ' ?'):
        dataset['workclass'][i] = ' Private'
    if(dataset['native-country'][i] == ' ?'):
        dataset['native-country'][i] = ' United-States'
        
# Setelah dilakukan analisis di bawah ini, akurasi justru menurun karena persebaran data pada occupation merata
#     if(dataset['occupation'][i] == ' ?'):
#         dataset['occupation'][i] = ' Prof-specialty'

for i in dataset.columns.values :
    le.fit(dataset[i])
    dataset[i] = le.transform(dataset[i])      
    

# masing-masing instance di setiap negara memiliki fnwght yang mirip sehingga tidak diperlukan lagi
dataset = dataset.drop(['fnlwght'], 1)

# penghasilan/jam berbeda untuk setiap pekerjaan sehingga atribut hours-per-week dapat dihapus
dataset = dataset.drop(['hours-per-week'],1)

# education-num hanya merupakan representasi dari education sehingga tidak diperlukan lagi
dataset = dataset.drop(['education-num'], 1)


# Memisahkan atribut income sebagai target
target = np.array(dataset['income'])
data = np.array(dataset.drop(['income'],1))

print(dataset)

C:\Users\FX553VD\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\FX553VD\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


       age  workclass  education  marital-status  occupation  relationship  \
0       22          6          9               4           1             1   
1       33          5          9               2           4             0   
2       21          3         11               0           6             1   
3       36          3          1               2           6             0   
4       11          3          9               2          10             5   
5       20          3         12               2           4             5   
6       32          3          6               3           8             1   
7       35          5         11               2           4             0   
8       14          3         12               4          10             1   
9       25          3          9               2           4             0   
10      20          3         15               2           4             0   
11      13          6          9               2          10    

## Skenario eksperimen yang kami lakukan adalah dengan melakukan algoritma Naive Bayes, Decision Tree, KNN, dan MLP. Dari eksperimen tersebut akan diukur masing-masing akurasi yang dihasilkan.

# Eksperimen dengan menggunakan algoritma Naive Bayes, Decision Tree, KNN, dan MLP

## Naive Bayes

In [3]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

scoring = ['precision_macro', 'recall_macro']
nb = MultinomialNB()
scores = cross_val_score(nb, data, target, cv=10)
accuracy = scores.mean()

scores_2 = cross_validate(nb, data, target, scoring=scoring,
                 cv=10, return_train_score=False)
precision = scores_2['test_precision_macro']
recall = scores_2['test_recall_macro']
print("Akurasi : ", accuracy)
print("Presisi : ")
print(precision)
print("Recall : ")
print(recall)

Akurasi :  0.785295336297
Presisi : 
[ 0.70457577  0.71715605  0.69488332  0.70383346  0.70858675  0.72768013
  0.68326038  0.70302317  0.70101256  0.7361398 ]
Recall : 
[ 0.62331411  0.62524148  0.61806932  0.62374719  0.61976587  0.63564783
  0.60310581  0.62679562  0.61925195  0.64068795]


## Decision Tree

In [4]:
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

dt = tree.DecisionTreeClassifier()
scores = cross_val_score(dt, data, target, cv=10)
accuracy = scores.mean()
scores_2 = cross_validate(dt, data, target, scoring=scoring,
                 cv=10, return_train_score=False)
precision = scores_2['test_precision_macro']
recall = scores_2['test_recall_macro']
print("Akurasi : ", accuracy)
print("Presisi : ")
print(precision)
print("Recall : ")
print(recall)

Akurasi :  0.824636966006
Presisi : 
[ 0.75665773  0.76150819  0.7657695   0.75010823  0.76111737  0.76417774
  0.75799846  0.761614    0.76269716  0.76111219]
Recall : 
[ 0.74231546  0.74514976  0.75434251  0.73161655  0.75275535  0.75547768
  0.74547586  0.75252213  0.74749851  0.75905447]


## KNN

In [5]:
from sklearn.neighbors import KNeighborsClassifier

scoring = ['precision_macro', 'recall_macro']
neighbor = KNeighborsClassifier(n_neighbors=30, p=1)

scores = cross_val_score(neighbor, data, target, cv=10)
accuracy = scores.mean()
scores_2 = cross_validate(neighbor, data, target, scoring=scoring,
                 cv=10, return_train_score=False)
precision = scores_2['test_precision_macro']
recall = scores_2['test_recall_macro']
print("Akurasi : ", accuracy)
print("Presisi : ")
print(precision)
print("Recall : ")
print(recall)

Akurasi :  0.848622914999
Presisi : 
[ 0.78802692  0.80309604  0.81118619  0.79861071  0.80926519  0.80686715
  0.8101539   0.81575112  0.80292922  0.8119444 ]
Recall : 
[ 0.74237086  0.75274503  0.75322799  0.75025593  0.76144038  0.7632133
  0.75238797  0.76560952  0.74908981  0.77106862]


## MLP

In [6]:
from sklearn.neural_network import MLPClassifier


scoring = ['precision_macro', 'recall_macro']
mlp = MLPClassifier(activation='logistic', solver='sgd', learning_rate='constant',
                        learning_rate_init=0.1, max_iter=100)

score = cross_val_score(mlp, data, target, cv=10)
scores_2 = cross_validate(mlp, data, target, scoring=scoring,
                 cv=10, return_train_score=False)
# Menampilkan kinerja
print(">>> Kinerja NaiveBayes 10-fold cross ")
for i in range(10):
    print("Fold - " + str(i + 1) + " :", "%0.2f" % score[i])
print()
print("Akurasi: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))
print("Presisi : ")
print(precision)
print("Recall : ")
print(recall)

>>> Kinerja NaiveBayes 10-fold cross 
Fold - 1 : 0.82
Fold - 2 : 0.81
Fold - 3 : 0.81
Fold - 4 : 0.81
Fold - 5 : 0.79
Fold - 6 : 0.82
Fold - 7 : 0.80
Fold - 8 : 0.81
Fold - 9 : 0.81
Fold - 10 : 0.79

Akurasi: 0.81 (+/- 0.02)
Presisi : 
[ 0.78802692  0.80309604  0.81118619  0.79861071  0.80926519  0.80686715
  0.8101539   0.81575112  0.80292922  0.8119444 ]
Recall : 
[ 0.74237086  0.75274503  0.75322799  0.75025593  0.76144038  0.7632133
  0.75238797  0.76560952  0.74908981  0.77106862]


## Model terbaik yang didapatkan adalah KNN
## Analisis akurasi dan Confusion Matrix untuk KNN

In [7]:
from sklearn.metrics import confusion_matrix
import numpy as np

neighbor = KNeighborsClassifier(n_neighbors=30, p=1)
scores = cross_val_score(neighbor, data, target, cv=10)
accuracy = scores.mean()
scores_2 = cross_validate(neighbor, data, target, scoring=scoring,
                 cv=10, return_train_score=False)
precision = scores_2['test_precision_macro']
recall = scores_2['test_recall_macro']
print("Akurasi : ", accuracy)
neighbor.fit(data, target)
prediksi = []
prediksi = neighbor.predict(data)
print(prediksi)
print(target)
print(confusion_matrix(prediksi, target))

Akurasi :  0.848622914999
[0 1 0 ..., 0 0 1]
[0 0 0 ..., 0 0 1]
[[23271  3165]
 [ 1449  4676]]


## Memilih algoritma pembelajaran terbaik
### Dari ke empat algoritma tersebut, dipilihlah algoritma Decison Tree untuk dilakukan full training

In [8]:
from sklearn.externals import joblib

model = KNeighborsClassifier(n_neighbors=30, p=1)
model.fit(data, target)
joblib.dump(model, 'model_income.pkl')

['model_income.pkl']

## Membaca Model/Hipotesis dari File Eksternal

In [9]:
model = joblib.load('model_income.pkl')

## Load data test

In [11]:
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from pandas import read_csv 

dataset_test = read_csv('CencusIncome.test.txt', index_col=False,
                names=['age', 'workclass', 'fnlwght', 'education', 'education-num', 'marital-status',
                                                          'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
                                                          'hours-per-week', 'native-country', 'income'])

le = preprocessing.LabelEncoder()
for i in dataset_test.columns.values :
    le.fit(dataset_test[i])
    dataset_test[i] = le.transform(dataset_test[i])
    
dataset_test = dataset_test.drop(['hours-per-week'],1)
dataset_test = dataset_test.drop(['fnlwght'], 1)
dataset_test = dataset_test.drop(['education-num'], 1)
    
target_test = np.array(dataset_test['income'])
data_test = np.array(dataset_test.drop(['income'],1))
predict_test = []
for i in data_test : 
    predict_test.append(model.predict([i]))
accuracy = accuracy_score(target_test, predict_test)
print("akurasi : ", accuracy)


akurasi :  0.840611756035
